# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
from api_keys import ow_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [2]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

648

In [3]:
cities[0:15]
# lats[0:15]
# lngs[0:15]

['punta arenas',
 'tighina',
 'saldanha',
 'cabuyaro',
 'san patricio',
 'rikitea',
 'shenkursk',
 'kapaa',
 'hithadhoo',
 'srednekolymsk',
 'esperance',
 'mys shmidta',
 'ushuaia',
 'centralia',
 'mahebourg']

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [4]:
q=f"http://api.openweathermap.org/data/2.5/weather?id=524901&lang=en&appid={ow_key}"
requests.get(q).json()

{'coord': {'lon': 37.6156, 'lat': 55.7522},
 'weather': [{'id': 501,
   'main': 'Rain',
   'description': 'moderate rain',
   'icon': '10d'}],
 'base': 'stations',
 'main': {'temp': 276.47,
  'feels_like': 271.91,
  'temp_min': 276.15,
  'temp_max': 277.04,
  'pressure': 1006,
  'humidity': 87},
 'visibility': 9000,
 'wind': {'speed': 6, 'deg': 290},
 'rain': {'1h': 1.78},
 'clouds': {'all': 90},
 'dt': 1619536333,
 'sys': {'type': 1,
  'id': 9027,
  'country': 'RU',
  'sunrise': 1619488526,
  'sunset': 1619542718},
 'timezone': 10800,
 'id': 524901,
 'name': 'Moscow',
 'cod': 200}

In [11]:
# city = "austin"
city = "samusu"
q= f"http://api.openweathermap.org/data/2.5/weather?{city}&appid={ow_key}"
print(requests.get(q))
requests.get(q).json()['main']

<Response [400]>


KeyError: 'main'

In [15]:
ow_url = "http://api.openweathermap.org/data/2.5/weather?q="
units = "imperial"

weather_data = []
record = 1

print("Beginning Data Retrieval")
print("------------------------------")

for city in cities[0:15]:
        query_url = f"{ow_url}{city}&appid={ow_key}&units={units}"
        response = requests.get(query_url).json()
        try:
#             check = response['id']
            print(f"Processing Record {record:>2d} | City Id = {response['id']:<7d} [{city}]")
#             print(f"Processing Record {record:2d}: {city}")
            weather_data.append({
                'City': response['name'],
                'Lat': response['coord']['lat'],
                'Lng': response['coord']['lon'],
                'Max Temp': response['main']['temp_max'],
                'Humidity': response['main']['humidity'],
                'Cloudiness': response['clouds']['all'],
                'Wind Speed': response['wind']['speed'],
                'Country': response['sys']['country'],
                'Date': response['dt']
            })
            record += 1
        except KeyError:
            print(f"City [{city}] not found. Skipping...")
            pass

print("------------------------------")        
print("Data Retrieval Complete")
print("------------------------------")

Beginning Data Retrieval
------------------------------
Processing Record  1 | City Id = 3874787 [punta arenas]
Processing Record  2 | City Id = 618577  [tighina]
Processing Record  3 | City Id = 3361934 [saldanha]
Processing Record  4 | City Id = 3688151 [cabuyaro]
Processing Record  5 | City Id = 4726521 [san patricio]
Processing Record  6 | City Id = 4030556 [rikitea]
Processing Record  7 | City Id = 494884  [shenkursk]
Processing Record  8 | City Id = 5848280 [kapaa]
Processing Record  9 | City Id = 1282256 [hithadhoo]
Processing Record 10 | City Id = 2121025 [srednekolymsk]
Processing Record 11 | City Id = 2071860 [esperance]
City [mys shmidta] not found. Skipping...
Processing Record 12 | City Id = 3833367 [ushuaia]
Processing Record 13 | City Id = 5789683 [centralia]
Processing Record 14 | City Id = 934322  [mahebourg]
------------------------------
Data Retrieval Complete
------------------------------


In [16]:
response['name']

'Mahébourg'

In [17]:
len(weather_data)
# cities_weather

14

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [18]:
pd.DataFrame(data=weather_data)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Punta Arenas,-53.1500,-70.9167,46.40,93,90,5.75,CL,1619536721
1,Tighina,46.8306,29.4711,44.60,76,90,11.50,MD,1619536468
2,Saldanha,-33.0117,17.9442,69.80,64,97,5.75,ZA,1619536658
3,Cabuyaro,4.2817,-72.7940,76.30,89,100,6.96,CO,1619536721
4,San Patricio,28.0170,-97.5169,80.60,88,90,9.22,US,1619536721
5,Rikitea,-23.1203,-134.9692,76.17,75,42,8.34,PF,1619536722
6,Shenkursk,62.1091,42.8960,36.82,98,100,3.44,RU,1619536469
7,Kapaa,22.0752,-159.3190,66.20,94,1,4.61,US,1619536680
8,Hithadhoo,-0.6000,73.0833,83.32,63,96,1.12,MV,1619536470
9,Srednekolymsk,67.4500,153.6833,10.13,92,93,3.33,RU,1619536723


## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
#  Get the indices of cities that have humidity over 100%.


In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression